In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# imports
import numpy as np
import pandas as pd
import os
import PIL
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
#pip install --upgrade tensorflow

In [ ]:
#!pip uninstall tensorflow -y
#pip uninstall keras
# Try to switch the version of tensorflow and kera
#pip install tf-nightly
#pip install keras==2.6.0
#!pip install  tensorflow==2.6
#tf.config.set_visible_devices([], 'GPU')

In [ ]:
# CoAtNet
class MBConv(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides = 1, expand_ratio = 1, se_ratio = 4, residual = True, momentum = 0.9, epsilon = 0.01, convolution = tf.keras.layers.Conv2D, activation = tf.nn.swish, kernel_initializer = "he_normal", **kwargs):
        super(MBConv, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.expand_ratio = expand_ratio
        self.se_ratio = se_ratio
        self.residual = residual
        self.momentum = momentum
        self.epsilon = epsilon
        self.convolution = convolution
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        
    def build(self, input_shape):
        self.layers = []
        self.post = []
        if self.expand_ratio != 1:
            conv = self.convolution(input_shape[-1] * self.expand_ratio, 1, use_bias = False, kernel_initializer = self.kernel_initializer)
            norm = tf.keras.layers.BatchNormalization(momentum = self.momentum, epsilon = self.epsilon)
            act = tf.keras.layers.Activation(self.activation)
            input_shape = input_shape[:-1] + (input_shape[-1] * self.expand_ratio,)
            self.layers += [conv, norm, act]
        
        #Depthwise Convolution
        conv = self.convolution(input_shape[-1], self.kernel_size, strides = self.strides, groups = input_shape[-1], padding = "same", use_bias = False, kernel_initializer = self.kernel_initializer)
        norm = tf.keras.layers.BatchNormalization(momentum = self.momentum, epsilon = self.epsilon)
        act = tf.keras.layers.Activation(self.activation)
        self.layers += [conv, norm, act]
        
        #Squeeze and Excitation layer, if desired
        axis = list(range(1, len(input_shape) - 1))
        gap = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis = axis, keepdims = True))
        squeeze = self.convolution(max(1, int(input_shape[-1] / self.se_ratio)), 1, use_bias = True, kernel_initializer = self.kernel_initializer)
        act = tf.keras.layers.Activation(self.activation)
        excitation = self.convolution(input_shape[-1], 1, use_bias = True, kernel_initializer = self.kernel_initializer)
        se = lambda x: x * tf.nn.sigmoid(excitation(act(squeeze(gap(x)))))
        self.layers += [se]
        
        #Output Phase
        conv = self.convolution(self.filters, 1, use_bias = False, kernel_initializer = self.kernel_initializer)
        norm = tf.keras.layers.BatchNormalization(momentum = self.momentum, epsilon = self.epsilon)
        self.layers += [conv, norm]
        
        #Residual
        if self.residual:
            if 1 < self.strides:
                pool = tf.keras.layers.MaxPool2D(pool_size = self.strides + 1, strides = self.strides, padding = "same")
                self.post.append(pool)
            if input_shape[-1] != self.filters:
                resample = self.convolution(self.filters, 1, use_bias = False, kernel_initializer = self.kernel_initializer)
                self.post.append(resample)
        
    def call(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
            
        if self.residual:
            for layer in self.post:
                x = layer(x)
            out = out + x
        return out
        
    def get_config(self):
        config = super(MBConv, self).get_config()
        config["filters"] = self.filters
        config["kernel_size"] = self.kernel_size
        config["expand_ratio"] = self.expand_ratio
        config["se_ratio"] = self.se_ratio
        config["residual"] = self.residual
        config["momentum"] = self.momentum
        config["epsilon"] = self.epsilon
        config["convolution"] = self.convolution
        config["activation"] = self.activation
        config["kernel_initializer"] = self.kernel_initializer
        return config

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, emb_dim = 768, n_head = 12, out_dim = None, relative_window_size = None, dropout_rate = 0., kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0, stddev = 0.01), **kwargs):
        #ScaledDotProductAttention
        super(MultiHeadSelfAttention, self).__init__(**kwargs)
        self.emb_dim = emb_dim
        self.n_head = n_head
        if emb_dim % n_head != 0:
            raise ValueError("Shoud be embedding dimension % number of heads = 0.")
        if out_dim is None:
            out_dim = self.emb_dim
        self.out_dim = out_dim
        if relative_window_size is not None and np.ndim(relative_window_size) == 0:
            relative_window_size = [relative_window_size, relative_window_size]
        self.relative_window_size = relative_window_size
        self.projection_dim = emb_dim // n_head
        self.dropout_rate = dropout_rate
        self.query = tf.keras.layers.Dense(emb_dim, kernel_initializer = kernel_initializer)
        self.key = tf.keras.layers.Dense(emb_dim, kernel_initializer = kernel_initializer)
        self.value = tf.keras.layers.Dense(emb_dim, kernel_initializer = kernel_initializer)
        self.combine = tf.keras.layers.Dense(out_dim, kernel_initializer = kernel_initializer)
        
    def build(self, input_shape):
        if self.relative_window_size is not None:
            self.relative_position_bias_table = self.add_weight("relative_position_bias_table", shape = [((2 * self.relative_window_size[0]) - 1) * ((2 * self.relative_window_size[1]) - 1), self.n_head], trainable = self.trainable)
            coords_h = np.arange(self.relative_window_size[0])
            coords_w = np.arange(self.relative_window_size[1])
            coords = np.stack(np.meshgrid(coords_h, coords_w, indexing = "ij")) #2, Wh, Ww
            coords = np.reshape(coords, [2, -1])
            relative_coords = np.expand_dims(coords, axis = -1) - np.expand_dims(coords, axis = -2) #2, Wh * Ww, Wh * Ww
            relative_coords = np.transpose(relative_coords, [1, 2, 0]) #Wh * Ww, Wh * Ww, 2
            relative_coords[:, :, 0] += self.relative_window_size[0] - 1 #shift to start from 0
            relative_coords[:, :, 1] += self.relative_window_size[1] - 1
            relative_coords[:, :, 0] *= 2 * self.relative_window_size[1] - 1
            relative_position_index = np.sum(relative_coords, -1)
            self.relative_position_index = tf.Variable(tf.convert_to_tensor(relative_position_index), trainable = False, name= "relative_position_index")
        
    def attention(self, query, key, value, relative_position_bias = None):
        score = tf.matmul(query, key, transpose_b = True)
        n_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(n_key)
        if relative_position_bias is not None:
            scaled_score = scaled_score + relative_position_bias
        weight = tf.nn.softmax(scaled_score, axis = -1)
        if 0 < self.dropout_rate:
            weight = tf.nn.dropout(weight, self.dropout_rate)
        out = tf.matmul(weight, value)
        return out
    
    def separate_head(self, x):
        out = tf.keras.layers.Reshape([-1, self.n_head, self.projection_dim])(x)
        out = tf.keras.layers.Permute([2, 1, 3])(out)
        return out
    
    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)
        
        query = self.separate_head(query)
        key = self.separate_head(key)
        value = self.separate_head(value)
        
        relative_position_bias = None
        if self.relative_window_size is not None:
            relative_position_bias = tf.gather(self.relative_position_bias_table, tf.reshape(self.relative_position_index, [-1]))
            relative_position_bias = tf.reshape(relative_position_bias, [self.relative_window_size[0] * self.relative_window_size[1], self.relative_window_size[0] * self.relative_window_size[1], -1]) #Wh * Ww,Wh * Ww, nH
            relative_position_bias = tf.transpose(relative_position_bias, [2, 0, 1]) #nH, Wh * Ww, Wh * Ww
            relative_position_bias = tf.expand_dims(relative_position_bias, axis = 0)
        attention = self.attention(query, key, value, relative_position_bias)
        attention = tf.keras.layers.Permute([2, 1, 3])(attention)
        attention = tf.keras.layers.Reshape([-1, self.emb_dim])(attention)
        
        out = self.combine(attention)
        return out
        
    def get_config(self):
        config = super(MultiHeadSelfAttention, self).get_config()
        config["emb_dim"] = self.emb_dim
        config["n_head"] = self.n_head
        config["out_dim"] = self.out_dim
        config["relative_window_size"] = self.relative_window_size
        config["projection_dim"] = self.projection_dim
        config["dropout_rate"] = self.dropout_rate
        return config
        
class ConvTransformer(tf.keras.layers.Layer):
    def __init__(self, emb_dim = 768, n_head = 12, strides = 1, out_dim = None, epsilon = 1e-5, dropout_rate = 0., activation = tf.keras.activations.gelu, kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0, stddev = 0.01), **kwargs):
        super(ConvTransformer, self).__init__(**kwargs)
        self.emb_dim = emb_dim
        self.n_head = n_head
        self.strides = strides
        self.out_dim = out_dim if out_dim is not None else emb_dim
        self.epsilon = epsilon
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        
    def build(self, input_shape):
        self.attention = []
        self.residual = []
        
        #Attention
        shape = input_shape[1:3]
        if 1 < self.strides:
            shape = np.divide(np.add(shape, (self.strides - 1)), self.strides).astype(int)
            pool = tf.keras.layers.MaxPool2D(pool_size = self.strides + 1, strides = self.strides, padding = "same")
            self.attention.append(pool)
            self.residual.append(pool)
        if input_shape[-1] != self.out_dim:
            resample = tf.keras.layers.Conv2D(self.out_dim, 1, padding = "same", use_bias = False, kernel_initializer = "he_normal")
            self.residual.append(resample)
        pre_reshape = tf.keras.layers.Reshape([-1, input_shape[-1]])
        mhsa = MultiHeadSelfAttention(emb_dim = self.emb_dim, n_head = self.n_head, out_dim = self.out_dim, relative_window_size = shape, dropout_rate = self.dropout_rate)
        post_reshape = tf.keras.layers.Reshape([*shape, self.out_dim])
        self.attention += [pre_reshape, mhsa, post_reshape]
        
        self.ffn = []
        #Feed Forward Network
        norm = tf.keras.layers.LayerNormalization(epsilon = self.epsilon)
        dense1 = tf.keras.layers.Dense(self.out_dim, kernel_initializer = self.kernel_initializer)
        act = tf.keras.layers.Activation(self.activation)
        dense2 = tf.keras.layers.Dense(self.out_dim, kernel_initializer = self.kernel_initializer)
        self.ffn = [norm, dense1, act, dense2]
    
    def call(self, inputs):
        out = inputs
        for layer in self.attention:
            out = layer(out)
        for layer in self.residual:
            inputs = layer(inputs)
        out = out + inputs
        
        for layer in self.ffn:
            out = layer(out)
        return out
        
    def get_config(self):
        config = super(ConvTransformer, self).get_config()
        config["emb_dim"] = self.emb_dim
        config["n_head"] = self.n_head
        config["strides"] = self.strides
        config["out_dim"] = self.out_dim
        config["epsilon"] = self.epsilon
        config["dropout_rate"] = self.dropout_rate
        config["activation"] = self.activation
        config["kernel_initializer"] = self.kernel_initializer
        return config
    
def coatnet(x, n_class = 1000, include_top = True, n_depth = [2, 2, 6, 14, 2], n_feature = [64, 96, 192, 384, 768], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu, name = ""):
    #block : S > Stem, C > MBConv, T > Transformer
    if 0 < len(name):
        name += "_"
    if isinstance(stage_stride_size, int):
        stage_stride_size = [stage_stride_size] * len(block)
        
    out = x
    for i, (_n_depth, _n_feature, _block, _stage_stride_size) in enumerate(zip(n_depth, n_feature, block, stage_stride_size)):
        for j in range(_n_depth):
            stride_size = 1 if j != 0 else _stage_stride_size
            residual = out
            if _block.upper() == "C":# i == 0:
                out = tf.keras.layers.Conv2D(_n_feature, 1 if i != 0 else 3, strides = stride_size, padding = "same", use_bias = False, kernel_initializer = "he_normal", name = "{0}stage{1}_conv{2}".format(name, i, j + 1))(out)
                out = tf.keras.layers.BatchNormalization(momentum = 0.9, epsilon = 1e-5, name = "{0}stage{1}_norm{2}".format(name, i, j + 1))(out)
                out = tf.keras.layers.Activation(activation, name = "{0}stage{1}_act{2}".format(name, i, j + 1))(out)
            elif _block.upper() == "M":
                out = tf.keras.layers.BatchNormalization(momentum = 0.9, epsilon = 1e-5, name = "{0}stage{1}_pre_norm{2}".format(name, i, j + 1))(out)
                out = MBConv(_n_feature, 3, strides = stride_size, expand_ratio = expand_ratio, se_ratio = se_ratio, residual = True, momentum = 0.9, epsilon = 1e-5, activation = activation, name = "{0}stage{1}_mbconv{2}".format(name, i, j + 1))(out)
            elif _block.upper() == "T":
                out = tf.keras.layers.LayerNormalization(epsilon = 1e-5, name = "{0}stage{1}_pre_norm{2}".format(name, i, j + 1))(out)
                out = ConvTransformer(32 * 8, 8, strides = stride_size, out_dim = _n_feature, epsilon = 1e-5, activation = activation, name = "{0}stage{1}_transformer{2}".format(name, i, j + 1))(out)

    if include_top:
        out = tf.keras.layers.GlobalAveragePooling2D(name = "{0}gap".format(name))(out)
        if 0 < dropout_rate:
            out = tf.keras.layers.Dropout(dropout_rate, name = "{0}dropout".format(name))(out)
        out = tf.keras.layers.Dense(n_class, kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0, stddev = 0.01), name = "{0}logits".format(name))(out)
    return out

def coatnet0(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 3, 5, 2], n_feature = [64, 96, 192, 384, 768], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet1(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 6, 14, 2], n_feature = [64, 96, 192, 384, 768], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet2(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 6, 14, 2], n_feature = [128, 128, 256, 512, 1024], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet3(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 6, 14, 2], n_feature = [192, 192, 384, 768, 1536], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet4(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 12, 28, 2], n_feature = [192, 192, 384, 768, 1536], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet5(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 12, 28, 2], n_feature = [192, 256, 512, 1280, 2048], block = ["C", "M", "M", "T", "T"], stage_stride_size = 2, expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet6(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 4, 8, 42, 2], n_feature = [192, 192, 384, 768, 1536, 2048], block = ["C", "M", "M", "M", "T", "T"], stage_stride_size = [2, 2, 2, 2, 1, 2], expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

def coatnet7(input_tensor = None, input_shape = None, classes = 1000, include_top = True, weights = None):
    if input_tensor is None:
        img_input = tf.keras.layers.Input(shape = input_shape)
    else:
        if not tf.keras.backend.is_keras_tensor(input_tensor):
            img_input = tf.keras.layers.Input(tensor = input_tensor, shape = input_shape)
        else:
            img_input = input_tensor

    out = coatnet(img_input, classes, include_top, n_depth = [2, 2, 4, 8, 42, 2], n_feature = [192, 256, 512, 1024, 2048, 3072], block = ["C", "M", "M", "M", "T", "T"], stage_stride_size = [2, 2, 2, 2, 1, 2], expand_ratio = 4, se_ratio = 4, dropout_rate = 0., activation = tf.keras.activations.gelu)
    model = tf.keras.Model(img_input, out)
    
    if weights is not None:
        model.load_weights(weights)
    return model

In [ ]:
train_df = pd.read_csv('/content/drive/Shareddrives/519 project/train_COVIDx9A.txt', sep=" ", header=None)

#Columns are added because it was seen that column names were 0,1,2,3, so new column names are added
#which are given in descriptions
train_df.columns=['patient id', 'filename', 'class', 'data source']

# Since we are doing image classification, patient id and data source is of no importance to us, so
#we cn drop them
train_df=train_df.drop(['patient id', 'data source'], axis=1 )

In [ ]:
train_df.head()

,filename,class
0,SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0....,pneumonia
1,SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5....,pneumonia
2,SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10...,pneumonia
3,SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day...,pneumonia
4,SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day...,pneumonia


In [ ]:
test_df = pd.read_csv('/content/drive/Shareddrives/519 project/test_COVIDx9A.txt', sep=" ", header=None)
test_df.columns=['id', 'filename', 'class', 'data source' ]
test_df=test_df.drop(['id', 'data source'], axis=1 )

In [ ]:
test_df.head()

,filename,class
0,MIDRC-RICORD-1C-419639-003251-46647-0.png,COVID-19
1,MIDRC-RICORD-1C-419639-001464-39871-0.png,COVID-19
2,MIDRC-RICORD-1C-419639-000918-78965-0.png,COVID-19
3,MIDRC-RICORD-1C-419639-003318-64285-0.png,COVID-19
4,MIDRC-RICORD-1C-419639-001015-81591-0.png,COVID-19


In [ ]:
train_df['class'].value_counts()

COVID-19     16490
normal        8085
pneumonia     5555
Name: class, dtype: int64

In [ ]:
covid  = train_df[train_df['class']=='COVID-19']   #negative values in class column
normal = train_df[train_df['class']=='normal']  #positive values in class column
pneumonia = train_df[train_df['class']=='pneumonia']
from sklearn.utils import resample
#majority class that  is negative, we need to downsample/decrease that class so that there is no bias
#n_samples = 2158 means we want 2158 sample of class negative, since there are 2158 samples of class positive
df_covid_downsampled = resample(covid, replace = True, n_samples = 5555) 
df_normal_downsampled = resample(normal, replace = True, n_samples = 5555)
#concatenate
train_df = pd.concat([pneumonia, df_covid_downsampled,df_normal_downsampled])

from sklearn.utils import shuffle
train_df = shuffle(train_df) # shuffling so that there is particular sequence

In [ ]:
train_df['class'].value_counts()

pneumonia    5555
COVID-19     5555
normal       5555
Name: class, dtype: int64

In [ ]:
train_data, valid_data = train_test_split(train_df, train_size=0.8, random_state=0)

In [ ]:
train_path = '/content/drive/Shareddrives/519 project/train_30000/'  #directory path
test_path = '/content/drive/Shareddrives/519 project/test.zip (Unzipped Files)'

In [ ]:
#train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   #shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
train_datagen = ImageDataGenerator(rescale = 1./255., shear_range = 0.2)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

#Now fit the them to get the images from directory (name of the images are given in dataframe) with augmentation


train_gen = train_datagen.flow_from_dataframe(dataframe = train_data, directory=train_path, x_col='filename', 
                                              y_col='class', target_size=(224,224), batch_size=16, 
                                               classes=['COVID-19','normal','pneumonia'],class_mode='sparse')
valid_gen = test_datagen.flow_from_dataframe(dataframe = valid_data, directory=train_path, x_col='filename',
                                             y_col='class', target_size=(224,224), batch_size=16, 
                                            classes=['COVID-19','normal','pneumonia'],class_mode='sparse')
test_gen = test_datagen.flow_from_dataframe(dataframe = test_df, directory=test_path, x_col='filename', 
                                            y_col='class', target_size=(224,224), batch_size=1,
                                             classes=['COVID-19','normal','pneumonia'],class_mode='sparse')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 10712 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 2620 validated image filenames belonging to 3 classes.
Found 650 validated image filenames belonging to 3 classes.
Found 400 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2683 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
train_gen1 = train_datagen.flow_from_dataframe(dataframe = train_data, directory=train_path, x_col='filename', 
                                              y_col='class', target_size=(224,224), batch_size=4, 
                                               classes=['COVID-19','normal','pneumonia'],class_mode='sparse')
valid_gen1 = test_datagen.flow_from_dataframe(dataframe = valid_data, directory=train_path, x_col='filename',
                                             y_col='class', target_size=(224,224), batch_size=4, 
                                            classes=['COVID-19','normal','pneumonia'],class_mode='sparse')
test_gen1 = test_datagen.flow_from_dataframe(dataframe = test_df, directory=test_path, x_col='filename', 
                                            y_col='class', target_size=(224,224), batch_size=1,
                                             classes=['COVID-19','normal','pneumonia'],class_mode='sparse')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 10712 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 2620 validated image filenames belonging to 3 classes.
Found 650 validated image filenames belonging to 3 classes.
Found 400 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2683 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
model = coatnet0(input_shape = (224, 224, 3), include_top = False)
flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
dense = tf.keras.layers.Dense(128, activation = "relu")(flatten)
drop_out = tf.keras.layers.Dropout(0.2)(dense)
prediction = tf.keras.layers.Dense(3, activation = "softmax", name = "prediction")(drop_out)
model = tf.keras.Model(model.input, prediction)

In [ ]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(7e-6)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
weights = compute_class_weight(class_weight = "balanced", classes = np.unique(train_gen.classes), y = train_gen.classes)
cw = dict(zip(np.unique(train_gen.classes), weights))
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h1", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', mode = "min", verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

model.compile(optimizer = opt, loss = loss,
              metrics=['accuracy'])

In [ ]:
model.fit(train_gen, validation_data = valid_gen, epochs = 15, class_weight=cw, callbacks=callbacks)

Epoch 1/15
167/167 [==============================] - 1654s 10s/step - loss: 0.8823 - accuracy: 0.6032 - val_loss: 0.7200 - val_accuracy: 0.6813 - lr: 7.0000e-06
Epoch 2/15
167/167 [==============================] - 146s 873ms/step - loss: 0.6584 - accuracy: 0.7231 - val_loss: 0.5107 - val_accuracy: 0.7932 - lr: 7.0000e-06
Epoch 3/15
167/167 [==============================] - 149s 892ms/step - loss: 0.5627 - accuracy: 0.7737 - val_loss: 0.4761 - val_accuracy: 0.8003 - lr: 7.0000e-06
Epoch 4/15
167/167 [==============================] - 146s 874ms/step - loss: 0.5251 - accuracy: 0.7842 - val_loss: 0.4909 - val_accuracy: 0.7918 - lr: 7.0000e-06
Epoch 5/15
167/167 [==============================] - ETA: 0s - loss: 0.4954 - accuracy: 0.7891
Epoch 5: ReduceLROnPlateau reducing learning rate to 3.5000000480067683e-06.
167/167 [==============================] - 146s 870ms/step - loss: 0.4954 - accuracy: 0.7891 - val_loss: 0.4899 - val_accuracy: 0.8017 - lr: 7.0000e-06
Epoch 6/15
167/167 [====

In [ ]:
# ResNet: 12 Epochs; Train_acc: 0.9674; Val_acc:0.9215; Test_acc:0.8550
model.evaluate(test_gen)

400/400 [==============================] - 338s 843ms/step - loss: 0.3364 - accuracy: 0.8650


[0.33641260862350464, 0.8650000095367432]

* CoAtNet0: 15 Epochs; Train_acc: 0.865; Val_acc:0.848; Test_acc:0.865

In [ ]:
#model.save_weights("covid_classifier_model.h6")

In [ ]:
#model.load_weights("covid_classifier_model.h6")

In [ ]:
#model.evaluate(test_gen)

* CoAtNet1: 15 Epochs; Train_acc: 0.862; Val_acc:0.854; Test_acc:0.8525

In [ ]:
model = coatnet1(input_shape = (224, 224, 3), include_top = False)
flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
dense = tf.keras.layers.Dense(128, activation = "relu")(flatten)
drop_out = tf.keras.layers.Dropout(0.2)(dense)
prediction = tf.keras.layers.Dense(3, activation = "softmax", name = "prediction")(drop_out)
model = tf.keras.Model(model.input, prediction)
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(7e-6)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
weights = compute_class_weight(class_weight = "balanced", classes = np.unique(train_gen.classes), y = train_gen.classes)
cw = dict(zip(np.unique(train_gen.classes), weights))
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h1", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', mode = "min", verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

model.compile(optimizer = opt, loss = loss,
              metrics=['accuracy'])


In [ ]:
model.fit(train_gen, validation_data = valid_gen, epochs = 15, class_weight=cw, callbacks=callbacks)

Epoch 1/15
167/167 [==============================] - 278s 1s/step - loss: 0.9068 - accuracy: 0.5530 - val_loss: 0.7108 - val_accuracy: 0.6870 - lr: 7.0000e-06
Epoch 2/15
167/167 [==============================] - 237s 1s/step - loss: 0.7353 - accuracy: 0.6901 - val_loss: 0.6997 - val_accuracy: 0.7025 - lr: 7.0000e-06
Epoch 3/15
167/167 [==============================] - 237s 1s/step - loss: 0.6434 - accuracy: 0.7340 - val_loss: 0.5732 - val_accuracy: 0.7465 - lr: 7.0000e-06
Epoch 4/15
167/167 [==============================] - 237s 1s/step - loss: 0.5974 - accuracy: 0.7448 - val_loss: 0.5378 - val_accuracy: 0.8116 - lr: 7.0000e-06
Epoch 5/15
167/167 [==============================] - 237s 1s/step - loss: 0.5446 - accuracy: 0.7673 - val_loss: 0.4744 - val_accuracy: 0.8201 - lr: 7.0000e-06
Epoch 6/15
167/167 [==============================] - 237s 1s/step - loss: 0.5428 - accuracy: 0.7718 - val_loss: 0.4779 - val_accuracy: 0.8130 - lr: 7.0000e-06
Epoch 7/15
167/167 [====================

In [ ]:
model.evaluate(test_gen)

400/400 [==============================] - 48s 120ms/step - loss: 0.3897 - accuracy: 0.8525


[0.38969185948371887, 0.8525000214576721]

* CoAtNet2: 15 Epochs; Train_acc: 0.851; Val_acc:0.839; Test_acc:0.832

In [ ]:
model = coatnet2(input_shape = (224, 224, 3), include_top = False)
flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
dense = tf.keras.layers.Dense(128, activation = "relu")(flatten)
drop_out = tf.keras.layers.Dropout(0.2)(dense)
prediction = tf.keras.layers.Dense(3, activation = "softmax", name = "prediction")(drop_out)
model = tf.keras.Model(model.input, prediction)
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(7e-6)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
weights = compute_class_weight(class_weight = "balanced", classes = np.unique(train_gen.classes), y = train_gen.classes)
cw = dict(zip(np.unique(train_gen.classes), weights))
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h1", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', mode = "min", verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

model.compile(optimizer = opt, loss = loss,
              metrics=['accuracy'])

In [ ]:
model.fit(train_gen1, validation_data = valid_gen1, epochs = 15, class_weight=cw, callbacks=callbacks)

Epoch 1/15
334/334 [==============================] - 468s 1s/step - loss: 0.8472 - accuracy: 0.5946 - val_loss: 0.7863 - val_accuracy: 0.6161 - lr: 7.0000e-06
Epoch 2/15
334/334 [==============================] - 367s 1s/step - loss: 0.7587 - accuracy: 0.6501 - val_loss: 0.7115 - val_accuracy: 0.6728 - lr: 7.0000e-06
Epoch 3/15
334/334 [==============================] - 369s 1s/step - loss: 0.7160 - accuracy: 0.6778 - val_loss: 0.7502 - val_accuracy: 0.6586 - lr: 7.0000e-06
Epoch 4/15
334/334 [==============================] - 368s 1s/step - loss: 0.6939 - accuracy: 0.6793 - val_loss: 0.6369 - val_accuracy: 0.7380 - lr: 7.0000e-06
Epoch 5/15
334/334 [==============================] - 369s 1s/step - loss: 0.7037 - accuracy: 0.6860 - val_loss: 0.6839 - val_accuracy: 0.6714 - lr: 7.0000e-06
Epoch 6/15
334/334 [==============================] - ETA: 0s - loss: 0.7323 - accuracy: 0.6699
Epoch 6: ReduceLROnPlateau reducing learning rate to 3.5000000480067683e-06.
334/334 [==================

In [ ]:
model.evaluate(test_gen)

* CoAtNet3: 15 Epochs; Train_acc: 0.; Val_acc:0.; Test_acc:0.

In [ ]:
model = coatnet3(input_shape = (224, 224, 3), include_top = False)
flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
dense = tf.keras.layers.Dense(128, activation = "relu")(flatten)
drop_out = tf.keras.layers.Dropout(0.2)(dense)
prediction = tf.keras.layers.Dense(3, activation = "softmax", name = "prediction")(drop_out)
model = tf.keras.Model(model.input, prediction)
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(7e-6)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
weights = compute_class_weight(class_weight = "balanced", classes = np.unique(train_gen.classes), y = train_gen.classes)
cw = dict(zip(np.unique(train_gen.classes), weights))
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h1", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', mode = "min", verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

model.compile(optimizer = opt, loss = loss,
              metrics=['accuracy'])

In [ ]:
model.fit(train_gen1, validation_data = valid_gen1, epochs = 15, class_weight=cw, callbacks=callbacks)

Epoch 1/15
655/655 [==============================] - 1068s 2s/step - loss: 0.9575 - accuracy: 0.5168 - val_loss: 0.8483 - val_accuracy: 0.5785 - lr: 7.0000e-06
Epoch 2/15
655/655 [==============================] - 622s 949ms/step - loss: 0.8956 - accuracy: 0.5397 - val_loss: 0.8097 - val_accuracy: 0.5831 - lr: 7.0000e-06
Epoch 3/15
655/655 [==============================] - 619s 945ms/step - loss: 0.8844 - accuracy: 0.5630 - val_loss: 0.8155 - val_accuracy: 0.6015 - lr: 7.0000e-06
Epoch 4/15
655/655 [==============================] - ETA: 0s - loss: 0.8797 - accuracy: 0.5542
Epoch 4: ReduceLROnPlateau reducing learning rate to 3.5000000480067683e-06.
655/655 [==============================] - 619s 945ms/step - loss: 0.8797 - accuracy: 0.5542 - val_loss: 0.8663 - val_accuracy: 0.5323 - lr: 7.0000e-06
Epoch 5/15
655/655 [==============================] - 618s 944ms/step - loss: 0.9102 - accuracy: 0.5607 - val_loss: 0.8411 - val_accuracy: 0.6123 - lr: 3.5000e-06
Epoch 5: early stopping


In [ ]:
model.evaluate(test_gen1)

400/400 [==============================] - 194s 485ms/step - loss: 0.7206 - accuracy: 0.6100


[0.7205551266670227, 0.6100000143051147]

* CoAtNet4: 15 Epochs; Train_acc: 0.; Val_acc:0.; Test_acc:0.

In [ ]:
model = coatnet4(input_shape = (224, 224, 3), include_top = False)
flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
dense = tf.keras.layers.Dense(128, activation = "relu")(flatten)
drop_out = tf.keras.layers.Dropout(0.2)(dense)
prediction = tf.keras.layers.Dense(3, activation = "softmax", name = "prediction")(drop_out)
model = tf.keras.Model(model.input, prediction)
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(1e-5)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
weights = compute_class_weight(class_weight = "balanced", classes = np.unique(train_gen.classes), y = train_gen.classes)
cw = dict(zip(np.unique(train_gen.classes), weights))
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("covid_classifier_model.h1", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', mode = "min", verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

model.compile(optimizer = opt, loss = loss,
              metrics=['accuracy'])


In [ ]:
model.fit(train_gen1, validation_data = valid_gen1, epochs = 15, class_weight=cw, callbacks=callbacks)

Epoch 1/15
655/655 [==============================] - 1079s 2s/step - loss: 1.1059 - accuracy: 0.3584 - val_loss: 1.0977 - val_accuracy: 0.3477 - lr: 1.0000e-05
Epoch 2/15
655/655 [==============================] - 991s 2s/step - loss: 1.1009 - accuracy: 0.3603 - val_loss: 1.0980 - val_accuracy: 0.3477 - lr: 1.0000e-05
Epoch 3/15
655/655 [==============================] - ETA: 0s - loss: 1.1009 - accuracy: 0.3137
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
655/655 [==============================] - 987s 2s/step - loss: 1.1009 - accuracy: 0.3137 - val_loss: 1.0986 - val_accuracy: 0.2600 - lr: 1.0000e-05
Epoch 4/15
655/655 [==============================] - 978s 1s/step - loss: 1.0986 - accuracy: 0.2527 - val_loss: 1.0986 - val_accuracy: 0.2600 - lr: 5.0000e-06
Epoch 4: early stopping


In [ ]:
model.evaluate(test_gen1)

400/400 [==============================] - 88s 219ms/step - loss: 1.0987 - accuracy: 0.2500


[1.0986504554748535, 0.25]